In [4]:
import pandas as pd
import backtrader as bt
import matplotlib.pyplot as plt
%matplotlib inline

In [37]:
lob=pd.read_csv('../datasets/lob_sample_signals.csv') #(1.3--1.6)
tape=pd.read_csv('../datasets/Tapes_all.csv')

In [38]:
# 设置第一列为索引
lob.set_index('Datetime', inplace=True)

tape['Datetime'] = pd.to_datetime(tape['Date']) + pd.to_timedelta(tape['Timestamp'], unit='s')

# Set the new 'Datetime' column as the index
tape.set_index('Datetime', inplace=True)

# Drop the original 'Timestamp' and 'Date' columns as they are no longer needed
tape.drop(['Timestamp', 'Date'], axis=1, inplace=True)

# Show the first few rows of the dataframe to confirm changes
tape=tape['2025-01-02':'2025-01-06']

In [39]:
# 假设lob_df和tape_df是两个Pandas DataFrame，分别包含LOB数据和tape数据
# 同时假设这两个DataFrame的索引都是以时间戳形式存在的
if not isinstance(lob.index, pd.DatetimeIndex):
    lob.index = pd.to_datetime(lob.index)

if not isinstance(tape.index, pd.DatetimeIndex):
    tape.index = pd.to_datetime(tape.index)
# 使用merge_asof进行非精确时间戳合并
# 这里假设'lob_df'和'tape_df'已经正确设置了以时间为索引
merged_df = pd.merge_asof(lob, tape, left_index=True, right_index=True, tolerance=pd.Timedelta('1sec'), direction='nearest')

In [40]:
# check the result
merged_df

# reset the index
merged_df.reset_index(inplace=True)
merged_df.head()
# save the merged dataframe to a new csv file
# merged_df.to_csv('../datasets/lob_tape_merged.csv')

,Datetime,Total Bid Quantity,Total Ask Quantity,Max Bid Price,Min Ask Price,Spread,Weighted Avg Bid Price,Weighted Avg Ask Price,Bid-Ask Quantity Ratio,Buy_Signal_Spread,Sell_Signal_Spread,Buy_Signal_BidAskRatio,Sell_Signal_BidAskRatio,Buy_Signal_LargeOrder,Sell_Signal_LargeOrder,Weighted_Avg_Price,Price_Volatility,Price,Volume
0,2025-01-02 00:00:01.333,6.0,1.0,1.0,800.0,799.0,1.000000,800.0,6.0,0,1,1,0,0,0,400.500000,NaN,NaN,NaN
1,2025-01-02 00:00:01.581,6.0,1.0,1.0,799.0,798.0,1.000000,799.0,6.0,0,1,1,0,0,0,400.000000,NaN,NaN,NaN
2,2025-01-02 00:00:01.643,6.0,1.0,1.0,798.0,797.0,1.000000,798.0,6.0,0,1,1,0,0,0,399.500000,NaN,NaN,NaN
3,2025-01-02 00:00:01.736,7.0,1.0,261.0,798.0,537.0,38.142857,798.0,7.0,0,1,1,0,0,0,418.071429,NaN,NaN,NaN
4,2025-01-02 00:00:01.984,7.0,1.0,261.0,797.0,536.0,38.142857,797.0,7.0,0,1,1,0,0,0,417.571429,NaN,NaN,NaN


In [54]:
# fill the na in the merged dataframe for the 'Price'  column
merged_df['Price'].fillna(method='bfill', inplace=True) # 使用后向填充的方式填充缺失值，即使用前一个非缺失值填充缺失值

# merged_df.isnull().sum()

# save the merged dataframe to a new csv file
# merged_df.to_csv('../datasets/lob_tape_merged_ffill.csv', index=False)

In [5]:
class CustomData(bt.feeds.GenericCSVData):
    lines = ('max_bid', 'min_ask',  
             'buy_signal_spread', 'sell_signal_spread',
             'buy_signal_bidaskratio', 'sell_signal_bidaskratio',
             'buy_signal_largeorder', 'sell_signal_largeorder') 
    
    params = (
        ('datetime', 0),
        ('close', 17),  # 使用Tape数据的Price作为close
        ('volume', 18),  # 使用Tape数据的Volume作为volume
        ('max_bid', 3),  # 加载Max Bid Price
        ('min_ask', 4),  # 加载Min Ask Price
        ('buy_signal_spread', 9),  # 'Buy_Signal_Spread'列，根据实际情况调整索引
        ('sell_signal_spread', 10),  # 'Sell_Signal_Spread'列，根据实际情况调整索引
        ('buy_signal_bidaskratio', 11),  # 'Buy_Signal_BidAskRatio'列，根据实际情况调整索引
        ('sell_signal_bidaskratio', 12),  # 'Sell_Signal_BidAskRatio'列，根据实际情况调整索引
        ('buy_signal_largeorder', 13),  # 'Buy_Signal_LargeOrder'列，根据实际情况调整索引
        ('sell_signal_largeorder', 14),  # 'Sell_Signal_LargeOrder'列，根据实际情况调整索引
        ('dtformat', '%Y-%m-%d %H:%M:%S.%f'),
        ('timeframe', bt.TimeFrame.Ticks),
    )


In [6]:
class MyStrategy(bt.Strategy):
    def log(self, txt, dt=None):
        ''' 日志函数，用于记录策略执行过程中的重要信息 '''
        dt = dt or self.datas[0].datetime.datetime(0)
        print(f'{dt.isoformat()}, {txt}')
        
    def __init__(self):
        # 初始化order（订单状态），buyprice（买入价格）和buycomm（买入佣金）
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.data_close = self.datas[0].close  # 使用Price作为交易决策的参考
        
        # 初始化信号
        self.buy_signal_spread = self.datas[0].buy_signal_spread
        self.sell_signal_spread = self.datas[0].sell_signal_spread
        self.buy_signal_bidaskratio = self.datas[0].buy_signal_bidaskratio
        self.sell_signal_bidaskratio = self.datas[0].sell_signal_bidaskratio
        self.buy_signal_largeorder = self.datas[0].buy_signal_largeorder
        self.sell_signal_largeorder = self.datas[0].sell_signal_largeorder
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
 
        # 若order状态为Completed，则进行下一步判断
        if order.status in [order.Completed]:
            # 订单已经完成，若为买入，则记录买入价格price，总花费cost（value），佣金comm
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                # 更新买入价格和佣金
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            
            # 若为卖出，则记录卖出价格price，总收入value，佣金comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
 
            self.bar_executed = len(self)
        
        # 若order状态为Canceled/Margin/Rejected，则记录订单取消/保证金不足/拒绝
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
 
        # 重置订单状态
        self.order = None
    
    # 一个trade是一次完整的买入和卖出过程，该函数用于记录交易的盈利情况
    def notify_trade(self, trade):
        # 若trade不是空且trade不是已经关闭 则退出该函数
        if not trade.isclosed:
            return
        # 若trade已经关闭，则记录交易的毛利润pnl和净利润pnlcomm
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        if self.order:  # 检查是否有指令等待执行
            return
        
        if not self.position:  # 如果当前没有持仓
            if self.buy_signal_spread[0] > 0 and self.buy_signal_bidaskratio[0] > 0:
                self.log('BUY CREATE, %.2f' % self.data_close[0])
                self.order = self.buy()  # 执行买入指令
            
        else:
            if self.sell_signal_spread[0] > 0 and self.sell_signal_bidaskratio[0] > 0:
                self.log('SELL CREATE, %.2f' % self.data_close[0])
                self.order = self.sell()  # 执行卖出指令

In [7]:
# 加载数据
data = CustomData(dataname='../datasets/lob_tape_merged_ffill.csv')

# 初始化cerebro回测系统设置
cerebro = bt.Cerebro()

# 将数据传入回测系统
cerebro.adddata(data)

# 将策略加载到回测系统中
cerebro.addstrategy(MyStrategy)

# 设置初始资本为10,000
cerebro.broker.setcash(1000000.0)

# 设置交易手续费为千分之一
# cerebro.broker.setcommission(commission=0.001)

# # 设置每笔交易的股票数目
# cerebro.addsizer(bt.sizers.FixedSize, stake=10)

# 在开始时输出账户价值
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# 运行回测系统
cerebro.run()

# 在结束时输出账户价值
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


Starting Portfolio Value: 1000000.00
2025-01-02T00:01:22.367004, BUY CREATE, 267.00
2025-01-02T00:01:22.429003, BUY EXECUTED, Price: 35.00, Cost: 35.00, Comm 0.00
2025-01-02T00:01:56.652998, SELL CREATE, 265.00
2025-01-02T00:01:57.025003, SELL EXECUTED, Price: 15.00, Cost: 35.00, Comm 0.00
2025-01-02T00:01:57.025003, OPERATION PROFIT, GROSS -20.00, NET -20.00
2025-01-02T00:02:05.550002, BUY CREATE, 266.00
2025-01-02T00:02:05.705001, BUY EXECUTED, Price: 59.00, Cost: 59.00, Comm 0.00
2025-01-02T00:04:02.792004, SELL CREATE, 261.00
2025-01-02T00:04:02.885003, SELL EXECUTED, Price: 21.00, Cost: 59.00, Comm 0.00
2025-01-02T00:04:02.885003, OPERATION PROFIT, GROSS -38.00, NET -38.00
2025-01-02T00:04:58.592003, BUY CREATE, 269.00
2025-01-02T00:04:58.809000, BUY EXECUTED, Price: 50.00, Cost: 50.00, Comm 0.00
2025-01-02T00:08:05.025995, SELL CREATE, 259.00
2025-01-02T00:08:05.150003, SELL EXECUTED, Price: 22.00, Cost: 50.00, Comm 0.00
2025-01-02T00:08:05.150003, OPERATION PROFIT, GROSS -28.00,